In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import cudf, glob, gc, pickle

VER = 166

In [2]:
files = glob.glob('../../data/train_data/train_parquet/*')
len( files )

100

In [3]:
files += glob.glob('../../data/train_data/test_parquet/*')
len( files )

120

In [4]:
files[:4]

['/raid/Kaggle/otto/valid/train_parquet/099.parquet',
 '/raid/Kaggle/otto/valid/train_parquet/047.parquet',
 '/raid/Kaggle/otto/valid/train_parquet/043.parquet',
 '/raid/Kaggle/otto/valid/train_parquet/086.parquet']

In [5]:
#type_weight = {0:0, 1:1, 2:1}

In [6]:
#MN = 1661119200

In [7]:
%%time
PIECES = 1
SIZE = 1.86e6/PIECES

# COMPUTE IN PARTS FOR MEMORY MANGEMENT
for PART in range(PIECES):
    print()
    print('### PART',PART+1)
    
    # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
    # => OUTER CHUNKS
    for a,b in [(0,20),(20,40),(40,60),(60,80),(80,100),(100,120)]:
        print(f'Processing {b-a} files...')
        
        # => INNER CHUNKS
        for k in range(a,b):
            # READ FILE
            df = cudf.read_parquet(files[k])
            
            df['x'] = df.groupby('session').aid.transform('count')
            if a>=100:
                df = df.loc[df.x>3]
            else:
                df = df.loc[df.x>6]
            df = df.drop('x',axis=1)
            
            #df = df.loc[df.ts>1662328791 - 60*60*24*28]
            #df = df.loc[( df.ts >= MN + 60*60*24*DAY )] # START ON DAY OF WEEK 
            #if len(df)==0: continue
                
            df = df.sort_values(['session','ts'],ascending=[True,False])

            # CREATE PAIRS
            df = df.merge(df.loc[df['type'].isin([1,2])], on='session')
            #df = df.loc[ ((df.ts_y - df.ts_x)> 0) ] #& ( df.ts_x < MN + 60*60*24*(DAY+1) ) ] 
            # MEMORY MANAGEMENT COMPUTE IN PARTS
            df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
            # ASSIGN WEIGHTS
            df = df[['session', 'aid_x', 'aid_y','ts_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            w = (1/2)**((df.ts_y - df.ts_x).abs()/60/60) # 6 HOUR HALF LIFE
            df['wgt'] = w #df.type_y.map(type_weight)*w
            
            df = df[['aid_x','aid_y','wgt']]

            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x','aid_y']).wgt.sum()
            # COMBINE INNER CHUNKS
            if k==a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k,', ',end='')
        print()
        # COMBINE OUTER CHUNKS
        if a==0: tmp = tmp2
        else: tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    # CONVERT MATRIX TO DICTIONARY
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    # SAVE TOP 40
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n',axis=1)
    # SAVE PART TO DISK
    df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
    with open(f'../../data/covisit_matrices/top_40_aids_v{VER}_{PART}.pkl', 'wb') as f:
        pickle.dump(df.to_dict(), f)
        


### PART 1
Processing 20 files...
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 
Processing 20 files...
20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 
Processing 20 files...
40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 
Processing 20 files...
60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 
Processing 20 files...
80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 
Processing 20 files...
100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 
CPU times: user 1min 34s, sys: 42.1 s, total: 2min 16s
Wall time: 2min 21s
